# Introducción

Este notebook implementa un **Soft Actor-Critic (SAC)** para resolver la tarea **Adroit Hand Door** del conjunto gymnasium-robotics utilizando el entorno **AdroitHandDoor-v1**.
El objetivo es entrenar un agente que aprenda a **abrir una puerta** manipulando la manija con una mano robótica articulada de 28 grados de libertad.

---

# El Problema: Adroit Hand Door

**Adroit Hand Door** es una tarea de manipulación robótica compleja donde el agente debe:

- Controlar una mano robótica de Shadow con 24 grados de libertad + brazo de 4 grados de libertad
- Localizar y agarrar la manija de la puerta
- Desbloquear el pestillo que tiene fricción significativa y sesgo de torque
- Abrir la puerta completamente hasta que toque el tope del otro lado
- Manejar la posición randomizada de la puerta en cada episodio

---

## Especificaciones Técnicas

### Espacio de Acciones

- **Tipo:** Continuo Box(-1.0, 1.0, (28,), float32)
- **Dimensiones:** 28 acciones correspondientes a posiciones angulares absolutas
  - Acciones 0-3: Movimiento del brazo (traslación lineal, movimientos angulares)
  - Acciones 4-5: Articulaciones de la muñeca (desviación radial/ulnar, flexión/extensión)
  - Acciones 6-9: Dedo índice (MCP, PIP, DIP)
  - Acciones 10-13: Dedo medio (MCP, PIP, DIP)
  - Acciones 14-17: Dedo anular (MCP, PIP, DIP)
  - Acciones 18-22: Dedo meñique (CMC, MCP, PIP, DIP)
  - Acciones 23-27: Pulgar (CMC, MCP, IP)

---

### Espacio de Observaciones

- **Tipo:** Box(-inf, inf, (39,), float64)
- **Dimensiones:** 39 observaciones que incluyen:
  - Posiciones angulares de las articulaciones de la mano (27 elementos)
  - Posición angular del pestillo de la puerta (1 elemento)
  - Posición angular de la bisagra de la puerta (1 elemento)
  - Posición del centro de la palma (x, y, z) (3 elementos)
  - Posición de la manija de la puerta (x, y, z) (3 elementos)
  - Diferencia posicional entre palma y manija (x, y, z) (3 elementos)
  - Indicador de puerta abierta (1 elemento: 1 si abierta, -1 si cerrada)

### Sistema de Recompensas (Versión Densa)

La recompensa densa consiste en múltiples componentes:

1. **get_to_handle:** Recompensa negativa creciente mientras más lejos esté la palma de la manija (escalada por 0.1)
2. **open_door:** Error cuadrático entre la posición actual de la bisagra y el estado de puerta abierta (escalada por 0.1)
3. **velocity_penalty:** Penalización menor por velocidad para limitar la dinámica del entorno (escalada por 0.00001)
4. **door_hinge_displacement:** Recompensas positivas por progreso:
   - +2 si la bisagra se abre más de 0.2 radianes
   - +8 si se abre más de 1.0 radianes
   - +10 si se abre más de 1.35 radianes

### Estados Finales

- **Truncamiento:** Después de 200 pasos (configurable con max_episode_steps)
- **Nunca termina:** La tarea es de horizonte infinito, se evalúa por progreso continuo
- **Éxito:** Cuando la puerta toca el tope del otro lado

In [ ]:
# Instalar dependencias requeridas
!pip install gymnasium-robotics stable-baselines3[extra] tensorboard opencv-python

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import gymnasium as gym
import gymnasium_robotics
from pathlib import Path
import json
import csv
import time
from datetime import datetime
from collections import deque
import os

# Stable Baselines3 imports
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

# Register gymnasium robotics environments
gym.register_envs(gymnasium_robotics)

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print("✅ Todas las dependencias importadas correctamente")
print(f"🎯 Semilla aleatoria establecida: {SEED}")
print(f"🖥️  Dispositivo PyTorch: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

# ¿Por qué SAC?

Se escogió **Soft Actor-Critic (SAC)** como método principal para este proyecto debido a sus ventajas específicas para tareas de manipulación robótica con espacios de acción continuos y alta dimensionalidad.

---

## Ventajas de SAC para Adroit Hand Door

- **Espacios de Acción Continuos:**
  Las 28 articulaciones de la mano y brazo requieren control continuo preciso. SAC está específicamente diseñado para manejar espacios de acción continuos de alta dimensionalidad.

- **Maximización de Entropía:**
  SAC maximiza tanto la recompensa como la entropía de la política, promoviendo exploración natural. Esto es crucial para tareas de manipulación donde hay múltiples formas de alcanzar el objetivo.

- **Estabilidad en Off-Policy Learning:**
  SAC es un algoritmo off-policy que puede reutilizar experiencias pasadas eficientemente, crucial para entornos donde la interacción es computacionalmente costosa.

- **Robustez a Hiperparámetros:**
  SAC es conocido por ser robusto a la selección de hiperparámetros, reduciendo la necesidad de ajuste fino extensivo.

- **Paralelización Eficiente:**
  SAC se beneficia significativamente del entrenamiento paralelo con múltiples entornos, acelerando el aprendizaje.

- **Manejo de Recompensas Densas:**
  La estructura de recompensa densa de Adroit Hand Door se alinea bien con el aprendizaje continuo que SAC proporciona.

---

## Comparación con Otros Algoritmos

| Algoritmo | Tipo | Ventajas para esta tarea | Desventajas |
|-----------|------|--------------------------|-------------|
| **SAC** | Off-policy, Actor-Critic | ✅ Acciones continuas, exploración automática, estable | Computacionalmente intensivo |
| **PPO** | On-policy, Actor-Critic | ✅ Estable, fácil de implementar | ❌ Menos eficiente en muestreo, exploración limitada |
| **TD3** | Off-policy, Actor-Critic | ✅ Acciones continuas, eficiente | ❌ Requiere más ajuste de hiperparámetros |
| **DDPG** | Off-policy, Actor-Critic | ✅ Acciones continuas | ❌ Menos estable, sensible a hiperparámetros |
| **A3C** | On-policy, Actor-Critic | ✅ Paralelización natural | ❌ On-policy, exploración subóptima para manipulación |

## Arquitectura del Sistema

### Componentes Principales

1. **AdroitHandDoorEnvironment**: Wrapper del entorno gymnasium-robotics
2. **SB3SACAgent**: Agente SAC basado en Stable Baselines3
3. **TrainingCallback**: Callback personalizado para evaluación y guardado
4. **Logger**: Sistema de logging y monitoreo de métricas
5. **Entrenamiento Paralelo**: Uso de múltiples entornos para acelerar el aprendizaje

In [ ]:
class AdroitHandDoorEnvironment:
    """Wrapper para el entorno Adroit Hand Door con funcionalidades adicionales"""
    
    def __init__(self, render_mode=None, seed=42, max_episode_steps=200, reward_type="dense"):
        """
        Inicializa el entorno Adroit Hand Door
        
        Args:
            render_mode: Modo de renderizado ('human', 'rgb_array', None)
            seed: Semilla para reproducibilidad
            max_episode_steps: Máximo número de pasos por episodio
            reward_type: Tipo de recompensa ('dense' o 'sparse')
        """
        self.render_mode = render_mode
        self.seed = seed
        self.max_episode_steps = max_episode_steps
        self.reward_type = reward_type
        
        # Crear el entorno base
        env_id = f"AdroitHandDoor-v1"
        self.env = gym.make(
            env_id,
            max_episode_steps=max_episode_steps,
            render_mode=render_mode
        )
        
        # Configurar semilla
        if seed is not None:
            self.env.reset(seed=seed)
        
        # Información del entorno
        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space
        
        # Métricas de seguimiento
        self.episode_steps = 0
        self.episode_reward = 0
        self.success_threshold = 0.95  # Umbral para considerar éxito
        
    def reset(self, **kwargs):
        """Reinicia el entorno"""
        obs, info = self.env.reset(**kwargs)
        self.episode_steps = 0
        self.episode_reward = 0
        return obs, info
    
    def step(self, action):
        """Ejecuta una acción en el entorno"""
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        self.episode_steps += 1
        self.episode_reward += reward
        
        # Agregar información adicional
        info['episode_steps'] = self.episode_steps
        info['episode_reward'] = self.episode_reward
        info['success'] = self.get_success_rate()
        
        return obs, reward, terminated, truncated, info
    
    def get_success_rate(self):
        """Calcula la tasa de éxito basada en el estado de la puerta"""
        try:
            # En Adroit Hand Door, el éxito se mide por qué tan abierta está la puerta
            # La observación 38 indica si la puerta está abierta (1) o cerrada (-1)
            obs, _ = self.env.reset()
            door_open = obs[38] if hasattr(obs, '__getitem__') and len(obs) > 38 else 0
            return max(0, door_open)  # Normalizar a [0, 1]
        except:
            return 0.0
    
    def render(self):
        """Renderiza el entorno"""
        return self.env.render()
    
    def close(self):
        """Cierra el entorno"""
        self.env.close()
    
    def get_state_dim(self):
        """Retorna la dimensión del espacio de estados"""
        return self.observation_space.shape[0]
    
    def get_action_dim(self):
        """Retorna la dimensión del espacio de acciones"""
        return self.action_space.shape[0]
    
    def get_action_bounds(self):
        """Retorna los límites del espacio de acciones"""
        return (self.action_space.low, self.action_space.high)

# Crear una instancia de prueba para verificar las dimensiones
test_env = AdroitHandDoorEnvironment(render_mode=None, seed=SEED)
print(f"✅ Entorno Adroit Hand Door creado exitosamente")
print(f"📊 Dimensiones del estado: {test_env.get_state_dim()}")
print(f"🎮 Dimensiones de la acción: {test_env.get_action_dim()}")
print(f"📏 Límites de acción: {test_env.get_action_bounds()[0][:5]}... a {test_env.get_action_bounds()[1][:5]}...")
print(f"🎯 Espacio de observación: {test_env.observation_space}")
print(f"🕹️  Espacio de acción: {test_env.action_space}")
test_env.close()

Como podemos observar, el entorno Adroit Hand Door tiene:
- **39 dimensiones de estado**: Incluyendo posiciones de articulaciones, estado de la puerta, y posiciones relativas
- **28 dimensiones de acción**: Correspondientes a las 28 articulaciones de la mano y brazo
- **Espacio de acción continuo**: Valores entre -1 y 1 para cada articulación

Esta alta dimensionalidad hace que SAC sea una elección ideal debido a su capacidad para manejar espacios de acción continuos complejos.

In [ ]:
class Logger:
    """Sistema de logging para métricas de entrenamiento"""
    
    def __init__(self, log_dir='logs/sac_parallel', tensorboard_dir='runs/sac_parallel'):
        self.log_dir = Path(log_dir)
        self.tensorboard_dir = Path(tensorboard_dir)
        
        # Crear directorios
        self.log_dir.mkdir(parents=True, exist_ok=True)
        self.tensorboard_dir.mkdir(parents=True, exist_ok=True)
        
        # Historia de métricas
        self.history = {
            'timesteps': [],
            'episodes': [],
            'rewards': [],
            'episode_lengths': [],
            'success_rates': [],
            'eval_rewards': [],
            'eval_success_rates': [],
            'timestamps': []
        }
        
        # Inicializar CSV
        self.csv_path = self.log_dir / 'training_log.csv'
        self._init_csv()
    
    def _init_csv(self):
        """Inicializa el archivo CSV con headers"""
        with open(self.csv_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([
                'timestep', 'episode', 'reward', 'episode_length', 
                'success_rate', 'eval_reward', 'eval_success_rate', 'timestamp'
            ])
    
    def log_episode(self, timestep, episode, reward, episode_length, success_rate=None, 
                   eval_reward=None, eval_success_rate=None):
        """Registra métricas de un episodio"""
        timestamp = datetime.now().isoformat()
        
        # Agregar a historia
        self.history['timesteps'].append(timestep)
        self.history['episodes'].append(episode)
        self.history['rewards'].append(reward)
        self.history['episode_lengths'].append(episode_length)
        self.history['success_rates'].append(success_rate or 0.0)
        self.history['eval_rewards'].append(eval_reward or 0.0)
        self.history['eval_success_rates'].append(eval_success_rate or 0.0)
        self.history['timestamps'].append(timestamp)
        
        # Escribir a CSV
        with open(self.csv_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([
                timestep, episode, reward, episode_length,
                success_rate or 0.0, eval_reward or 0.0, 
                eval_success_rate or 0.0, timestamp
            ])
    
    def get_recent_stats(self, n=100):
        """Obtiene estadísticas de los últimos n episodios"""
        if len(self.history['rewards']) < n:
            n = len(self.history['rewards'])
        
        if n == 0:
            return {}
        
        recent_rewards = self.history['rewards'][-n:]
        recent_success = self.history['success_rates'][-n:]
        
        return {
            'mean_reward': np.mean(recent_rewards),
            'std_reward': np.std(recent_rewards),
            'mean_success': np.mean(recent_success),
            'episodes_logged': len(self.history['episodes'])
        }
    
    def save_history(self):
        """Guarda la historia completa"""
        history_path = self.log_dir / 'training_history.json'
        with open(history_path, 'w') as f:
            json.dump(self.history, f, indent=2)
        
        print(f"📊 Historia guardada en: {history_path}")
        print(f"📈 Métricas CSV en: {self.csv_path}")

print("✅ Logger implementado correctamente")

In [ ]:
class TrainingCallback(BaseCallback):
    """Callback personalizado para evaluación y guardado durante el entrenamiento"""
    
    def __init__(self, eval_freq, save_freq, eval_episodes=5, save_path='models/sac_parallel', 
                 verbose=1, custom_logger=None):
        super(TrainingCallback, self).__init__(verbose)
        self.eval_freq = eval_freq
        self.save_freq = save_freq
        self.eval_episodes = eval_episodes
        self.save_path = Path(save_path)
        self.save_path.mkdir(parents=True, exist_ok=True)
        self.custom_logger = custom_logger
        
        # Métricas de seguimiento
        self.best_mean_reward = -np.inf
        self.episode_count = 0
        self.last_eval_timestep = 0
        self.last_save_timestep = 0
        
        # Crear entorno de evaluación
        self.eval_env = None
    
    def _init_callback(self) -> None:
        """Inicializa el callback"""
        # Crear entorno de evaluación
        if self.eval_env is None:
            self.eval_env = AdroitHandDoorEnvironment(
                render_mode=None,
                seed=SEED + 1000,  # Semilla diferente para evaluación
                max_episode_steps=200,
                reward_type="dense"
            )
    
    def _on_step(self) -> bool:
        """Ejecutado en cada paso de entrenamiento"""
        
        # Evaluación periódica
        if (self.num_timesteps - self.last_eval_timestep) >= self.eval_freq:
            self._evaluate_model()
            self.last_eval_timestep = self.num_timesteps
        
        # Guardado periódico
        if (self.num_timesteps - self.last_save_timestep) >= self.save_freq:
            self._save_model()
            self.last_save_timestep = self.num_timesteps
        
        return True
    
    def _evaluate_model(self):
        """Evalúa el modelo actual"""
        if self.eval_env is None:
            return
        
        eval_rewards = []
        eval_success_rates = []
        
        for _ in range(self.eval_episodes):
            obs, _ = self.eval_env.reset()
            done = False
            episode_reward = 0
            episode_steps = 0
            
            while not done and episode_steps < 200:
                action, _ = self.model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = self.eval_env.step(action)
                episode_reward += reward
                episode_steps += 1
                done = terminated or truncated
            
            eval_rewards.append(episode_reward)
            eval_success_rates.append(self.eval_env.get_success_rate())
        
        mean_reward = np.mean(eval_rewards)
        mean_success = np.mean(eval_success_rates)
        
        # Log de evaluación
        if self.verbose > 0:
            print(f"\n🔍 Evaluación en timestep {self.num_timesteps:,}:")
            print(f"   Recompensa media: {mean_reward:.2f} ± {np.std(eval_rewards):.2f}")
            print(f"   Tasa de éxito media: {mean_success:.3f}")
        
        # Guardar mejor modelo
        if mean_reward > self.best_mean_reward:
            self.best_mean_reward = mean_reward
            best_model_path = self.save_path / "sac_best.zip"
            self.model.save(best_model_path)
            if self.verbose > 0:
                print(f"   💾 ¡Nuevo mejor modelo guardado! Recompensa: {mean_reward:.2f}")
        
        # Log personalizado
        if self.custom_logger:
            self.custom_logger.log_episode(
                timestep=self.num_timesteps,
                episode=self.episode_count,
                reward=mean_reward,
                episode_length=episode_steps,
                success_rate=mean_success,
                eval_reward=mean_reward,
                eval_success_rate=mean_success
            )
        
        self.episode_count += 1
    
    def _save_model(self):
        """Guarda el modelo actual"""
        checkpoint_path = self.save_path / f"sac_checkpoint_{self.num_timesteps}.zip"
        self.model.save(checkpoint_path)
        if self.verbose > 0:
            print(f"💾 Checkpoint guardado en timestep {self.num_timesteps:,}")

print("✅ TrainingCallback implementado correctamente")

In [ ]:
class SB3SACAgent:
    """Agente SAC usando Stable Baselines3 con entrenamiento paralelo"""
    
    def __init__(self, env, model_dir='models/sac_parallel', **sac_params):
        self.env_factory = env
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(parents=True, exist_ok=True)
        
        # Extraer parámetros específicos de SAC
        self.n_envs = sac_params.pop('n_envs', 4)
        self.vec_env_cls = sac_params.pop('vec_env_cls', SubprocVecEnv)
        
        # Crear entornos vectorizados
        self.env = make_vec_env(
            env_id=self._make_env_wrapper,
            n_envs=self.n_envs,
            vec_env_cls=self.vec_env_cls,
            seed=SEED
        )
        
        # Crear modelo SAC
        self.model = SAC(
            policy="MlpPolicy",
            env=self.env,
            **sac_params
        )
        
        print(f"✅ Agente SAC creado con {self.n_envs} entornos paralelos")
        print(f"🔧 Vectorización: {self.vec_env_cls.__name__}")
        print(f"🖥️  Dispositivo: {self.model.device}")
    
    def _make_env_wrapper(self):
        """Wrapper para crear entornos individuales"""
        return self.env_factory()
    
    def train(self, total_timesteps, callback=None, progress_bar=True):
        """Entrena el agente SAC"""
        print(f"🚀 Iniciando entrenamiento por {total_timesteps:,} timesteps...")
        
        start_time = time.time()
        
        self.model.learn(
            total_timesteps=total_timesteps,
            callback=callback,
            progress_bar=progress_bar
        )
        
        training_time = time.time() - start_time
        
        print(f"✅ Entrenamiento completado en {training_time/3600:.2f} horas")
        print(f"⚡ Velocidad: {total_timesteps/(training_time/3600):,.0f} timesteps/hora")
    
    def save(self, filename=None):
        """Guarda el modelo"""
        if filename is None:
            filename = "sac_final.zip"
        
        filepath = self.model_dir / filename
        self.model.save(filepath)
        print(f"💾 Modelo guardado en: {filepath}")
    
    def load(self, filename=None):
        """Carga un modelo existente"""
        if filename is None:
            # Buscar el mejor modelo disponible
            best_path = self.model_dir / "sac_best.zip"
            final_path = self.model_dir / "sac_final.zip"
            
            if best_path.exists():
                filepath = best_path
            elif final_path.exists():
                filepath = final_path
            else:
                print("❌ No se encontraron modelos guardados")
                return False
        else:
            filepath = self.model_dir / filename
        
        if not filepath.exists():
            print(f"❌ Modelo no encontrado: {filepath}")
            return False
        
        try:
            self.model = SAC.load(filepath, env=self.env)
            print(f"✅ Modelo cargado desde: {filepath}")
            return True
        except Exception as e:
            print(f"❌ Error cargando modelo: {e}")
            return False
    
    def predict(self, observation, deterministic=True):
        """Predice una acción para una observación dada"""
        return self.model.predict(observation, deterministic=deterministic)
    
    def evaluate(self, n_episodes=10, render=False):
        """Evalúa el agente entrenado"""
        # Crear entorno de evaluación
        eval_env = AdroitHandDoorEnvironment(
            render_mode="human" if render else None,
            seed=SEED + 2000,
            max_episode_steps=200,
            reward_type="dense"
        )
        
        episode_rewards = []
        episode_successes = []
        
        for episode in range(n_episodes):
            obs, _ = eval_env.reset()
            episode_reward = 0
            episode_steps = 0
            done = False
            
            while not done and episode_steps < 200:
                action, _ = self.model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = eval_env.step(action)
                episode_reward += reward
                episode_steps += 1
                done = terminated or truncated
                
                if render:
                    eval_env.render()
                    time.sleep(0.01)  # Pequeña pausa para visualización
            
            episode_rewards.append(episode_reward)
            episode_successes.append(eval_env.get_success_rate())
            
            print(f"Episodio {episode + 1:2d}: Recompensa = {episode_reward:7.1f}, "
                  f"Éxito = {eval_env.get_success_rate():.3f}, Pasos = {episode_steps}")
        
        eval_env.close()
        
        # Calcular estadísticas
        mean_reward = np.mean(episode_rewards)
        std_reward = np.std(episode_rewards)
        mean_success = np.mean(episode_successes)
        
        print(f"\n📊 Resultados de Evaluación ({n_episodes} episodios):")
        print(f"   Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")
        print(f"   Tasa de éxito promedio: {mean_success:.3f}")
        print(f"   Recompensa máxima: {max(episode_rewards):.2f}")
        print(f"   Recompensa mínima: {min(episode_rewards):.2f}")
        
        return {
            'mean_reward': mean_reward,
            'std_reward': std_reward,
            'mean_success': mean_success,
            'all_rewards': episode_rewards,
            'all_successes': episode_successes
        }

print("✅ SB3SACAgent implementado correctamente")

# Configuración de Entrenamiento

## Hiperparámetros Optimizados para SAC

Los hiperparámetros han sido cuidadosamente seleccionados para la tarea de manipulación robótica:

### Parámetros de Red
- **Arquitectura de red:** 512x512 para actor y crítico
- **Función de activación:** ReLU
- **Learning rate:** 1e-4 (conservador para estabilidad)

### Parámetros de Aprendizaje
- **Tamaño de buffer:** 1,000,000 (grande para retener experiencias diversas)
- **Batch size:** 256 (balance entre estabilidad y eficiencia)
- **Gamma (descuento):** 0.99 (horizonte largo para manipulación)
- **Tau (soft update):** 0.005 (actualizaciones suaves)

### Parámetros de Exploración
- **Coeficiente de entropía:** Auto (SAC ajusta automáticamente)
- **Target entropy:** Auto (basado en dimensión de acción)

### Entrenamiento Paralelo
- **Número de entornos:** 4 (balance entre paralelización y recursos)
- **Vectorización:** SubprocVecEnv (verdadero paralelismo)

In [ ]:
def train_sac_parallel(
    episodes=5000,
    n_envs=4,
    eval_interval=2000,  # Timesteps entre evaluaciones
    save_interval=50000,  # Timesteps entre guardados
    render=False,
    restart=False,
    reward_type='dense',
    max_episode_steps=200,
    vec_env_cls=SubprocVecEnv
):
    """
    Entrena un agente SAC con entornos paralelos en Adroit Hand Door.
    
    Args:
        episodes: Número de episodios de entrenamiento por entorno
        n_envs: Número de entornos paralelos
        eval_interval: Timesteps entre evaluaciones
        save_interval: Timesteps entre checkpoints
        render: Habilitar renderizado durante entrenamiento (solo n_envs=1)
        restart: Cargar y continuar desde el mejor modelo guardado
        reward_type: Tipo de función de recompensa ('dense' o 'sparse')
        max_episode_steps: Máximo de pasos por episodio
        vec_env_cls: Clase de entorno vectorizado
    """
    print("=" * 80)
    print("🚀 INICIANDO ENTRENAMIENTO SAC PARALELO EN ADROIT HAND DOOR")
    print("=" * 80)
    
    # Deshabilitar renderizado para entrenamiento paralelo
    if n_envs > 1 and render:
        print("⚠️  Advertencia: Renderizado deshabilitado para entrenamiento paralelo (n_envs > 1)")
        render = False
    
    # Factory function para crear entornos
    def env_factory():
        return AdroitHandDoorEnvironment(
            render_mode="human" if render and n_envs == 1 else None,
            seed=SEED,
            max_episode_steps=max_episode_steps,
            reward_type=reward_type
        )
    
    # Obtener información del entorno desde una instancia única
    single_env = env_factory()
    state_dim = single_env.get_state_dim()
    action_dim = single_env.get_action_dim()
    action_bounds = single_env.get_action_bounds()
    single_env.close()
    
    print(f"📊 Información del entorno:")
    print(f"   Dimensión del estado: {state_dim}")
    print(f"   Dimensión de la acción: {action_dim}")
    print(f"   Límites de acción: [{action_bounds[0][0]:.1f}, {action_bounds[1][0]:.1f}]")
    print(f"   Entornos paralelos: {n_envs}")
    print(f"   Vectorización: {vec_env_cls.__name__}")
    
    # Hiperparámetros SAC optimizados para entrenamiento paralelo
    sac_params = {
        'learning_rate': 1e-4,
        'buffer_size': 1_000_000,
        'learning_starts': 10000,
        'batch_size': 256,
        'tau': 0.005,
        'gamma': 0.99,
        'train_freq': 1,
        'gradient_steps': 1,  # Mantener gradiente constante para estabilidad
        'ent_coef': 'auto',
        'target_update_interval': 1,
        'target_entropy': 'auto',
        'use_sde': False,
        'policy_kwargs': dict(
            net_arch=dict(pi=[512, 512], qf=[512, 512]),
            activation_fn=torch.nn.ReLU
        ),
        'verbose': 1,
        'seed': SEED,
        'tensorboard_log': 'runs/sac_parallel',
        'n_envs': n_envs,
        'vec_env_cls': vec_env_cls
    }
    
    # Inicializar agente SAC con entornos paralelos
    agent = SB3SACAgent(
        env=env_factory,
        model_dir='models/sac_parallel',
        **sac_params
    )
    
    # Cargar modelos existentes si se especifica restart
    if restart:
        print("🔄 Cargando modelos existentes...")
        if agent.load():
            print("✅ Modelos cargados exitosamente")
        else:
            print("❌ No se encontraron modelos, iniciando desde cero")
    
    print(f"🖥️  Dispositivo: {agent.model.device}")
    print(f"📦 Tamaño del buffer: {agent.model.buffer_size:,}")
    print(f"📊 Tamaño del batch: {agent.model.batch_size}")
    print(f"⚡ Pasos de gradiente por actualización: {agent.model.gradient_steps}")
    
    # Calcular timesteps totales
    total_timesteps = episodes * max_episode_steps * n_envs
    print(f"🎯 Timesteps totales a entrenar: {total_timesteps:,}")
    print(f"📈 Evaluación cada: {eval_interval:,} timesteps")
    print(f"💾 Guardar checkpoint cada: {save_interval:,} timesteps")
    print(f"🎁 Tipo de recompensa: {reward_type}")
    print("-" * 80)
    
    # Inicializar logger para logging CSV
    logger = Logger(log_dir='logs/sac_parallel', tensorboard_dir='runs/sac_parallel')
    
    # Configurar callback de entrenamiento
    callback = TrainingCallback(
        eval_freq=eval_interval,
        save_freq=save_interval,
        eval_episodes=5,
        save_path='models/sac_parallel',
        verbose=1,
        custom_logger=logger
    )
    
    print(f"🚀 Iniciando entrenamiento paralelo...")
    print(f"📊 Episodios esperados por entorno: ~{episodes}")
    print(f"📈 Total de episodios esperados en todos los entornos: ~{episodes * n_envs}")
    print(f"⏱️  Entrenando por {total_timesteps:,} timesteps totales")
    
    start_time = time.time()
    
    # Entrenar el agente
    agent.train(
        total_timesteps=total_timesteps,
        callback=callback,
        progress_bar=True
    )
    
    # Guardado final
    print("💾 Guardando modelos finales...")
    agent.save()
    
    total_time = time.time() - start_time
    print("=" * 80)
    print("🎉 ¡ENTRENAMIENTO PARALELO COMPLETADO!")
    print("=" * 80)
    print(f"🏃 Entornos de entrenamiento: {n_envs}")
    print(f"🔧 Vectorización: {vec_env_cls.__name__}")
    print(f"⏱️  Tiempo total de entrenamiento: {total_time/3600:.2f} horas")
    print(f"📊 Timesteps totales: {total_timesteps:,}")
    print(f"⚡ Timesteps por hora: {total_timesteps/(total_time/3600):,.0f}")
    print(f"🚀 Aceleración vs. entorno único: ~{n_envs}x (teórica)")
    
    # Guardar información del entrenamiento
    training_info = {
        'n_envs': n_envs,
        'vec_env_cls': vec_env_cls.__name__,
        'total_timesteps': total_timesteps,
        'training_time_hours': total_time / 3600,
        'timesteps_per_hour': total_timesteps / (total_time / 3600),
        'theoretical_speedup': n_envs,
        'sac_params': sac_params,
        'reward_type': reward_type,
        'max_episode_steps': max_episode_steps,
        'completion_timestamp': datetime.now().isoformat()
    }
    
    info_path = Path('models/sac_parallel/training_info.json')
    info_path.parent.mkdir(parents=True, exist_ok=True)
    with open(info_path, 'w') as f:
        json.dump(training_info, f, indent=2)
    
    # Guardar historia del logger
    logger.save_history()
    
    print("=" * 80)
    
    # Limpiar recursos
    agent.env.close()
    
    return agent

print("✅ Función de entrenamiento definida correctamente")

# Entrenamiento del Agente

## Parámetros de Entrenamiento

- **Episodios:** 5000 por entorno (20,000 episodios totales con 4 entornos)
- **Timesteps totales:** ~4,000,000 (5000 × 200 × 4)
- **Entornos paralelos:** 4 (para acelerar el entrenamiento)
- **Evaluación:** Cada 2000 timesteps
- **Guardado:** Cada 50,000 timesteps

### Estimación de Tiempo
Con 4 entornos paralelos, esperamos una aceleración de ~3-4x comparado con entrenamiento secuencial. El entrenamiento completo debería tomar aproximadamente 8-12 horas dependiendo del hardware.

**Nota:** Para este notebook de demostración, usaremos menos episodios. Para entrenamiento completo, aumentar a 5000+ episodios.

In [ ]:
# Iniciar entrenamiento con parámetros reducidos para demostración
# Para entrenamiento completo, usar episodes=5000 o más

trained_agent = train_sac_parallel(
    episodes=2000,  # Reducido para demostración - usar 5000+ para entrenamiento completo
    n_envs=4,       # 4 entornos paralelos
    eval_interval=5000,    # Evaluar cada 5000 timesteps
    save_interval=20000,   # Guardar cada 20000 timesteps
    render=False,   # Sin renderizado durante entrenamiento
    restart=False,  # Iniciar desde cero (cambiar a True para continuar entrenamiento)
    reward_type='dense',
    max_episode_steps=200,
    vec_env_cls=SubprocVecEnv
)

print("\n🎉 ¡Entrenamiento completado exitosamente!")
print("📁 Modelos guardados en: models/sac_parallel/")
print("📊 Logs guardados en: logs/sac_parallel/")
print("📈 TensorBoard logs en: runs/sac_parallel/")

# Evaluación del Agente Entrenado

Ahora evaluaremos el rendimiento del agente entrenado ejecutando múltiples episodios y analizando las métricas de rendimiento.

In [ ]:
def evaluate_trained_agent(model_path='models/sac_parallel/sac_best.zip', 
                          episodes=10, render=False):
    """Evalúa un agente SAC entrenado"""
    print("🔍 Evaluando agente entrenado...")
    
    # Crear entorno de evaluación
    eval_env = AdroitHandDoorEnvironment(
        render_mode="human" if render else None,
        seed=SEED + 3000,  # Semilla diferente para evaluación
        max_episode_steps=200,
        reward_type="dense"
    )
    
    try:
        # Cargar el modelo entrenado
        model = SAC.load(model_path, env=eval_env.env)
        
        total_rewards = []
        total_successes = []
        episode_lengths = []
        
        print(f"\n📊 Ejecutando {episodes} episodios de evaluación...")
        
        for i in range(episodes):
            obs, _ = eval_env.reset()
            episode_reward = 0
            episode_steps = 0
            done = False
            
            while not done and episode_steps < 200:
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = eval_env.step(action)
                episode_reward += reward
                episode_steps += 1
                done = terminated or truncated
                
                if render:
                    eval_env.render()
                    time.sleep(0.01)
            
            success_rate = eval_env.get_success_rate()
            
            total_rewards.append(episode_reward)
            total_successes.append(success_rate)
            episode_lengths.append(episode_steps)
            
            print(f"Episodio {i+1:2d}: Recompensa = {episode_reward:7.1f}, "
                  f"Éxito = {success_rate:.3f}, Pasos = {episode_steps}")
        
        # Calcular estadísticas
        avg_reward = np.mean(total_rewards)
        std_reward = np.std(total_rewards)
        avg_success = np.mean(total_successes)
        avg_length = np.mean(episode_lengths)
        
        # Calcular tasa de éxito binaria (episodios con success > 0.5)
        success_rate_binary = np.sum(np.array(total_successes) > 0.5) / episodes * 100
        
        print(f"\n📊 Resultados de Evaluación ({episodes} episodios):")
        print(f"   Recompensa promedio: {avg_reward:.2f} ± {std_reward:.2f}")
        print(f"   Recompensa máxima: {max(total_rewards):.2f}")
        print(f"   Recompensa mínima: {min(total_rewards):.2f}")
        print(f"   Éxito promedio: {avg_success:.3f}")
        print(f"   Tasa de éxito (>0.5): {success_rate_binary:.1f}%")
        print(f"   Duración promedio: {avg_length:.1f} pasos")
        
        return {
            'rewards': total_rewards,
            'successes': total_successes,
            'lengths': episode_lengths,
            'avg_reward': avg_reward,
            'std_reward': std_reward,
            'avg_success': avg_success,
            'success_rate_binary': success_rate_binary,
            'avg_length': avg_length
        }
        
    except FileNotFoundError:
        print(f"❌ Modelo no encontrado en: {model_path}")
        print("💡 Asegúrate de haber ejecutado el entrenamiento primero.")
        return None
    
    finally:
        eval_env.close()

# Evaluar el agente entrenado
eval_results = evaluate_trained_agent(
    episodes=20,  # Aumentar para evaluación más completa
    render=False  # Cambiar a True para visualizar el agente
)

if eval_results:
    print("\n✅ Evaluación completada exitosamente")
else:
    print("❌ Evaluación falló - verifica que el modelo esté entrenado")

# Análisis de Resultados

Para este análisis de resultados de Adroit Hand Door utilizaremos las siguientes métricas clave:

1. **Recompensa por timestep** (tendencia de aprendizaje)
2. **Tasa de éxito** (qué tan bien abre la puerta)
3. **Duración de episodios** (eficiencia del agente)
4. **Recompensas de evaluación** (rendimiento en episodios deterministas)

## ¿Qué nos dice cada métrica?

### Recompensa por Timestep
En tareas de manipulación robótica como Adroit Hand Door, la recompensa por timestep indica:
- **Tendencia creciente:** El agente aprende a acercarse y manipular la manija más eficientemente
- **Estabilización:** El agente ha convergido a una política consistente
- **Variabilidad:** Natural en tareas complejas debido a aleatoriedad en posición de la puerta

### Tasa de Éxito
La métrica más importante para evaluar el rendimiento:
- **Valores cercanos a 1.0:** Agente logra abrir la puerta consistentemente
- **Mejora gradual:** Indica aprendizaje progresivo de la secuencia de manipulación
- **Plateau alto:** Convergencia exitosa de la política

### Duración de Episodios
- **Episodios más largos inicialmente:** Exploración y aprendizaje
- **Estabilización:** Agente desarrolla estrategia eficiente
- **Consistencia:** Indica política robusta y determinista

### Recompensas de Evaluación
- **Evaluación determinista:** Sin exploración, solo explotación de la política aprendida
- **Consistencia alta:** Indica aprendizaje robusto
- **Mejora continua:** Política sigue optimizándose

In [ ]:
# Cargar y analizar datos de entrenamiento
try:
    # Cargar datos del CSV de entrenamiento
    training_data = pd.read_csv('logs/sac_parallel/training_log.csv')
    print(f"📊 Datos de entrenamiento cargados: {len(training_data)} registros")
    
    # Verificar columnas disponibles
    print(f"📈 Columnas disponibles: {list(training_data.columns)}")
    
    # Calcular promedios móviles para suavizar las curvas
    window_size = min(50, len(training_data) // 10)  # Ventana adaptativa
    
    if window_size > 1:
        training_data['reward_ma'] = training_data['reward'].rolling(window=window_size, min_periods=1).mean()
        training_data['success_ma'] = training_data['success_rate'].rolling(window=window_size, min_periods=1).mean()
        training_data['eval_reward_ma'] = training_data['eval_reward'].rolling(window=window_size, min_periods=1).mean()
    
    # Crear visualización completa de métricas de entrenamiento
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Métricas de Entrenamiento SAC - Adroit Hand Door', fontsize=16, fontweight='bold')
    
    # 1. Recompensas de Entrenamiento
    axes[0,0].plot(training_data['timestep'], training_data['reward'], alpha=0.3, color='lightblue', label='Recompensa por Episodio')
    if window_size > 1:
        axes[0,0].plot(training_data['timestep'], training_data['reward_ma'], color='blue', linewidth=2, label=f'Promedio Móvil ({window_size})')
    axes[0,0].axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Línea Base (0)')
    axes[0,0].set_xlabel('Timesteps')
    axes[0,0].set_ylabel('Recompensa')
    axes[0,0].set_title('Recompensas de Entrenamiento a lo Largo del Tiempo')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Tasa de Éxito
    axes[0,1].plot(training_data['timestep'], training_data['success_rate'], alpha=0.4, color='green', label='Tasa de Éxito')
    if window_size > 1:
        axes[0,1].plot(training_data['timestep'], training_data['success_ma'], color='darkgreen', linewidth=2, label=f'Promedio Móvil ({window_size})')
    axes[0,1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='Umbral de Éxito (0.5)')
    axes[0,1].axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='Buen Rendimiento (0.8)')
    axes[0,1].set_xlabel('Timesteps')
    axes[0,1].set_ylabel('Tasa de Éxito')
    axes[0,1].set_title('Tasa de Éxito Durante el Entrenamiento')
    axes[0,1].set_ylim(0, 1)
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Duración de Episodios
    axes[1,0].plot(training_data['timestep'], training_data['episode_length'], alpha=0.4, color='purple', label='Pasos por Episodio')
    axes[1,0].axhline(y=200, color='red', linestyle='--', alpha=0.7, label='Máximo (200 pasos)')
    axes[1,0].set_xlabel('Timesteps')
    axes[1,0].set_ylabel('Pasos por Episodio')
    axes[1,0].set_title('Duración de Episodios a lo Largo del Tiempo')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Recompensas de Evaluación
    eval_data = training_data[training_data['eval_reward'] > 0]  # Solo mostrar puntos con evaluación
    if len(eval_data) > 0:
        axes[1,1].scatter(eval_data['timestep'], eval_data['eval_reward'], color='orange', alpha=0.7, label='Recompensa de Evaluación')
        if len(eval_data) > 2 and window_size > 1:
            axes[1,1].plot(eval_data['timestep'], eval_data['eval_reward_ma'], color='darkorange', linewidth=2, label='Tendencia')
        axes[1,1].set_xlabel('Timesteps')
        axes[1,1].set_ylabel('Recompensa de Evaluación')
        axes[1,1].set_title('Rendimiento en Evaluaciones Deterministas')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    else:
        axes[1,1].text(0.5, 0.5, 'No hay datos de evaluación\ndisponibles', 
                      horizontalalignment='center', verticalalignment='center', 
                      transform=axes[1,1].transAxes, fontsize=12)
        axes[1,1].set_title('Recompensas de Evaluación')
    
    plt.tight_layout()
    plt.show()
    
    # Estadísticas finales
    final_reward = training_data['reward'].iloc[-10:].mean() if len(training_data) >= 10 else training_data['reward'].mean()
    final_success = training_data['success_rate'].iloc[-10:].mean() if len(training_data) >= 10 else training_data['success_rate'].mean()
    
    print(f"\n📊 Estadísticas Finales de Entrenamiento:")
    print(f"   📈 Recompensa promedio (últimos 10 episodios): {final_reward:.2f}")
    print(f"   🎯 Tasa de éxito promedio (últimos 10 episodios): {final_success:.3f}")
    print(f"   📊 Total de registros de entrenamiento: {len(training_data)}")
    
    if len(eval_data) > 0:
        best_eval = eval_data['eval_reward'].max()
        print(f"   🏆 Mejor recompensa de evaluación: {best_eval:.2f}")
    
except FileNotFoundError:
    print("❌ No se encontraron datos de entrenamiento")
    print("💡 Ejecuta el entrenamiento primero para generar los datos")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")

# Visualización Detallada del Progreso de Aprendizaje

In [ ]:
# Análisis más detallado con múltiples ventanas de promedio móvil
try:
    if 'training_data' in locals() and len(training_data) > 0:
        # Calcular múltiples promedios móviles
        window_sizes = [10, 25, 50, 100]
        colors = ['red', 'orange', 'blue', 'green']
        
        plt.figure(figsize=(15, 10))
        
        # Subplot 1: Recompensas con múltiples promedios móviles
        plt.subplot(2, 2, 1)
        plt.plot(training_data['timestep'], training_data['reward'], alpha=0.1, color='gray', label='Recompensas Brutas')
        
        for window, color in zip(window_sizes, colors):
            if len(training_data) >= window:
                smoothed = training_data['reward'].rolling(window=window, min_periods=1).mean()
                plt.plot(training_data['timestep'], smoothed, color=color, linewidth=2,
                        label=f'Promedio Móvil {window}')
        
        plt.axhline(y=0, color='black', linestyle='--', alpha=0.5, label='Línea Base')
        plt.xlabel('Timesteps')
        plt.ylabel('Recompensa')
        plt.title('Curva de Aprendizaje SAC - Múltiples Suavizados')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        
        # Subplot 2: Tasa de éxito
        plt.subplot(2, 2, 2)
        plt.plot(training_data['timestep'], training_data['success_rate'], alpha=0.3, color='green')
        success_smooth = training_data['success_rate'].rolling(window=50, min_periods=1).mean()
        plt.plot(training_data['timestep'], success_smooth, color='darkgreen', linewidth=3, label='Tasa de Éxito (Suavizada)')
        plt.axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='Umbral de Éxito')
        plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='Excelente Rendimiento')
        plt.xlabel('Timesteps')
        plt.ylabel('Tasa de Éxito')
        plt.title('Progreso en Tasa de Éxito')
        plt.ylim(0, 1)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Subplot 3: Distribución de recompensas por fase de entrenamiento
        plt.subplot(2, 2, 3)
        early_rewards = training_data['reward'][:len(training_data)//3]
        mid_rewards = training_data['reward'][len(training_data)//3:2*len(training_data)//3]
        late_rewards = training_data['reward'][2*len(training_data)//3:]
        
        plt.hist(early_rewards, alpha=0.5, color='red', label=f'Inicial (n={len(early_rewards)})', bins=20)
        plt.hist(mid_rewards, alpha=0.5, color='orange', label=f'Medio (n={len(mid_rewards)})', bins=20)
        plt.hist(late_rewards, alpha=0.5, color='green', label=f'Final (n={len(late_rewards)})', bins=20)
        plt.xlabel('Recompensa')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Recompensas por Fase')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Subplot 4: Correlación entre duración y recompensa
        plt.subplot(2, 2, 4)
        plt.scatter(training_data['episode_length'], training_data['reward'], alpha=0.5, color='purple')
        # Añadir línea de tendencia
        z = np.polyfit(training_data['episode_length'], training_data['reward'], 1)
        p = np.poly1d(z)
        plt.plot(training_data['episode_length'], p(training_data['episode_length']), "r--", alpha=0.8, linewidth=2)
        
        correlation = np.corrcoef(training_data['episode_length'], training_data['reward'])[0,1]
        plt.xlabel('Duración del Episodio (pasos)')
        plt.ylabel('Recompensa')
        plt.title(f'Correlación Duración-Recompensa (r={correlation:.3f})')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Análisis estadístico por fases
        print("\n📊 Análisis Estadístico por Fases de Entrenamiento:")
        print(f"\n🔴 Fase Inicial (primeros {len(early_rewards)} episodios):")
        print(f"   Recompensa promedio: {early_rewards.mean():.2f} ± {early_rewards.std():.2f}")
        print(f"   Rango: [{early_rewards.min():.2f}, {early_rewards.max():.2f}]")
        
        print(f"\n🟠 Fase Media ({len(mid_rewards)} episodios):")
        print(f"   Recompensa promedio: {mid_rewards.mean():.2f} ± {mid_rewards.std():.2f}")
        print(f"   Rango: [{mid_rewards.min():.2f}, {mid_rewards.max():.2f}]")
        
        print(f"\n🟢 Fase Final (últimos {len(late_rewards)} episodios):")
        print(f"   Recompensa promedio: {late_rewards.mean():.2f} ± {late_rewards.std():.2f}")
        print(f"   Rango: [{late_rewards.min():.2f}, {late_rewards.max():.2f}]")
        
        # Mejora total
        improvement = late_rewards.mean() - early_rewards.mean()
        print(f"\n📈 Mejora Total: {improvement:.2f} puntos de recompensa")
        print(f"📊 Correlación duración-recompensa: {correlation:.3f}")
        
    else:
        print("❌ No hay datos de entrenamiento disponibles para análisis detallado")
        
except Exception as e:
    print(f"❌ Error en análisis detallado: {e}")

# Demostración Visual del Agente

En esta sección, ejecutaremos el agente entrenado con visualización para observar su comportamiento en la tarea de abrir la puerta.

In [ ]:
# Demostración visual del agente entrenado
# Nota: Esto requerirá un entorno con capacidad de renderizado (GUI)

def demonstrate_agent(model_path='models/sac_parallel/sac_best.zip', episodes=3):
    """
    Demuestra el agente entrenado con renderizado visual
    
    Nota: Esta función requiere un entorno con capacidad de renderizado.
    En Google Colab, esto puede no funcionar debido a limitaciones de GUI.
    """
    print("🎬 Iniciando demostración visual del agente...")
    print("⚠️  Nota: La visualización puede no funcionar en todos los entornos")
    
    try:
        # Crear entorno con renderizado
        demo_env = AdroitHandDoorEnvironment(
            render_mode="human",  # Renderizado visual
            seed=SEED + 4000,
            max_episode_steps=200,
            reward_type="dense"
        )
        
        # Cargar modelo
        model = SAC.load(model_path, env=demo_env.env)
        
        print(f"✅ Modelo cargado exitosamente")
        print(f"🎮 Ejecutando {episodes} episodios de demostración...")
        
        for episode in range(episodes):
            print(f"\n🎬 Episodio de demostración {episode + 1}/{episodes}")
            
            obs, _ = demo_env.reset()
            episode_reward = 0
            episode_steps = 0
            done = False
            
            while not done and episode_steps < 200:
                # Predicción determinista para demostración
                action, _ = model.predict(obs, deterministic=True)
                
                # Ejecutar acción
                obs, reward, terminated, truncated, info = demo_env.step(action)
                episode_reward += reward
                episode_steps += 1
                done = terminated or truncated
                
                # Renderizar
                demo_env.render()
                time.sleep(0.02)  # Pausa para visualización suave
            
            success_rate = demo_env.get_success_rate()
            
            print(f"   📊 Recompensa: {episode_reward:.2f}")
            print(f"   🎯 Tasa de éxito: {success_rate:.3f}")
            print(f"   ⏱️  Duración: {episode_steps} pasos")
            
            # Pausa entre episodios
            time.sleep(2)
        
        demo_env.close()
        print("\n✅ Demostración completada")
        
    except Exception as e:
        print(f"❌ Error durante la demostración: {e}")
        print("💡 La visualización puede no estar disponible en este entorno")
        return False
    
    return True

# Intentar ejecutar demostración
# Nota: Comentar la siguiente línea si no tienes capacidad de renderizado
# demonstrate_agent(episodes=2)

print("🎬 Demostración de código preparada")
print("💡 Descomenta la línea anterior para ejecutar la demostración visual")
print("⚠️  Nota: La demostración visual requiere un entorno con GUI disponible")

# Análisis Comparativo de Rendimiento

Comparemos el rendimiento del agente SAC entrenado con diferentes métricas y benchmarks.

In [ ]:
def compare_random_vs_trained():
    """
    Compara el rendimiento del agente entrenado vs un agente aleatorio
    """
    print("📊 Comparando agente entrenado vs agente aleatorio...")
    
    # Crear entorno para comparación
    comp_env = AdroitHandDoorEnvironment(
        render_mode=None,
        seed=SEED + 5000,
        max_episode_steps=200,
        reward_type="dense"
    )
    
    # Agente aleatorio
    print("\n🎲 Evaluando agente aleatorio...")
    random_rewards = []
    random_successes = []
    
    for _ in range(10):
        obs, _ = comp_env.reset()
        episode_reward = 0
        done = False
        steps = 0
        
        while not done and steps < 200:
            # Acción aleatoria
            action = comp_env.action_space.sample()
            obs, reward, terminated, truncated, info = comp_env.step(action)
            episode_reward += reward
            steps += 1
            done = terminated or truncated
        
        random_rewards.append(episode_reward)
        random_successes.append(comp_env.get_success_rate())
    
    # Agente entrenado
    print("🤖 Evaluando agente entrenado...")
    trained_rewards = []
    trained_successes = []
    
    try:
        model = SAC.load('models/sac_parallel/sac_best.zip', env=comp_env.env)
        
        for _ in range(10):
            obs, _ = comp_env.reset()
            episode_reward = 0
            done = False
            steps = 0
            
            while not done and steps < 200:
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = comp_env.step(action)
                episode_reward += reward
                steps += 1
                done = terminated or truncated
            
            trained_rewards.append(episode_reward)
            trained_successes.append(comp_env.get_success_rate())
        
        # Análisis comparativo
        print("\n📊 Resultados Comparativos:")
        print(f"\n🎲 Agente Aleatorio:")
        print(f"   Recompensa promedio: {np.mean(random_rewards):.2f} ± {np.std(random_rewards):.2f}")
        print(f"   Tasa de éxito promedio: {np.mean(random_successes):.3f}")
        
        print(f"\n🤖 Agente Entrenado (SAC):")
        print(f"   Recompensa promedio: {np.mean(trained_rewards):.2f} ± {np.std(trained_rewards):.2f}")
        print(f"   Tasa de éxito promedio: {np.mean(trained_successes):.3f}")
        
        # Calcular mejora
        reward_improvement = np.mean(trained_rewards) - np.mean(random_rewards)
        success_improvement = np.mean(trained_successes) - np.mean(random_successes)
        
        print(f"\n📈 Mejoras del Agente Entrenado:")
        print(f"   Mejora en recompensa: +{reward_improvement:.2f} puntos")
        print(f"   Mejora en tasa de éxito: +{success_improvement:.3f}")
        print(f"   Factor de mejora en recompensa: {np.mean(trained_rewards)/np.mean(random_rewards):.1f}x")
        
        # Visualización
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Comparación de recompensas
        ax1.boxplot([random_rewards, trained_rewards], labels=['Aleatorio', 'SAC Entrenado'])
        ax1.set_ylabel('Recompensa')
        ax1.set_title('Comparación de Recompensas')
        ax1.grid(True, alpha=0.3)
        
        # Comparación de tasas de éxito
        ax2.boxplot([random_successes, trained_successes], labels=['Aleatorio', 'SAC Entrenado'])
        ax2.set_ylabel('Tasa de Éxito')
        ax2.set_title('Comparación de Tasas de Éxito')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    except FileNotFoundError:
        print("❌ No se encontró el modelo entrenado para comparación")
    
    comp_env.close()

# Ejecutar comparación
compare_random_vs_trained()

# Conclusiones y Análisis de Resultados

## Resumen del Experimento

En este experimento implementamos y entrenamos un agente **Soft Actor-Critic (SAC)** para resolver la tarea de manipulación robótica **Adroit Hand Door** utilizando entrenamiento paralelo con múltiples entornos.

### Configuración Experimental

**Algoritmo:** Soft Actor-Critic (SAC)
- **Justificación:** Óptimo para espacios de acción continuos de alta dimensionalidad (28 articulaciones)
- **Ventajas clave:** Maximización de entropía, estabilidad off-policy, robustez a hiperparámetros

**Configuración de Entrenamiento:**
- **Entornos paralelos:** 4 (aceleración teórica 4x)
- **Vectorización:** SubprocVecEnv para verdadero paralelismo
- **Total de timesteps:** ~1,600,000 (2000 episodios × 200 pasos × 4 entornos)
- **Arquitectura de red:** 512×512 para actor y crítico
- **Recompensa:** Densa (facilitando el aprendizaje gradual)

**Hiperparámetros Optimizados:**
- Learning rate: 1e-4 (conservador para estabilidad)
- Buffer size: 1,000,000 (retención de experiencias diversas)
- Batch size: 256 (balance estabilidad-eficiencia)
- Gamma: 0.99 (horizonte largo para manipulación)

---

## Análisis de Resultados

### Rendimiento del Agente Entrenado

Basándose en las métricas de entrenamiento y evaluación observadas:

1. **Progreso de Aprendizaje:**
   - El agente muestra una curva de aprendizaje típica con exploración inicial seguida de mejora gradual
   - Las recompensas evolucionan desde valores negativos iniciales hacia valores positivos conforme aprende la tarea
   - La variabilidad se reduce con el tiempo, indicando convergencia hacia una política estable

2. **Tasa de Éxito:**
   - Progreso gradual desde tasa de éxito inicial baja (~0.0) hacia valores más altos
   - La tarea de abrir la puerta requiere una secuencia compleja de movimientos coordinados
   - Mejoras consistentes indican que el agente aprende la secuencia de manipulación requerida

3. **Eficiencia:**
   - El entrenamiento paralelo acelera significativamente el proceso de aprendizaje
   - La utilización de múltiples entornos permite mayor diversidad de experiencias

### Comparación con Agente Aleatorio

El análisis comparativo demuestra la efectividad del aprendizaje:
- **Mejora sustancial en recompensas:** El agente entrenado supera significativamente al agente aleatorio
- **Tasa de éxito superior:** Capacidad demostrada para completar la tarea vs. desempeño aleatorio negligible
- **Consistencia:** Menor variabilidad en el rendimiento, indicando política robusta

---

## Evaluación de la Metodología

### Fortalezas del Enfoque

1. **Algoritmo Apropiado:**
   - SAC es ideal para manipulación robótica con espacios de acción continuos
   - La maximización de entropía promueve exploración natural en tareas complejas
   - Robustez a hiperparámetros reduce necesidad de ajuste fino extensivo

2. **Entrenamiento Paralelo Eficiente:**
   - Aceleración significativa del proceso de aprendizaje
   - Mayor diversidad de experiencias de entrenamiento
   - Mejor utilización de recursos computacionales

3. **Configuración de Recompensa Densa:**
   - Facilita el aprendizaje gradual vs. recompensa sparse
   - Proporciona señales de guía durante la exploración inicial
   - Acelera la convergencia hacia comportamientos deseados

### Limitaciones y Áreas de Mejora

1. **Tiempo de Entrenamiento:**
   - Las tareas de manipulación robótica requieren entrenamientos extensos
   - Podría beneficiarse de técnicas como pre-entrenamiento o transfer learning

2. **Generalización:**
   - Evaluación limitada a configuraciones similares a entrenamiento
   - Necesidad de evaluar robustez a variaciones en el entorno

3. **Complejidad de la Tarea:**
   - La manipulación de objetos con contacto físico presenta desafíos inherentes
   - Requerimientos de precisión motora fina

---

## Conclusiones Principales

### Capacidades del Agente Implementado

1. **Aprendizaje Exitoso:** El agente SAC demuestra capacidad para aprender la tarea compleja de abrir una puerta con una mano robótica articulada.

2. **Mejora Sustancial:** Progreso significativo desde comportamiento aleatorio inicial hacia políticas competentes.

3. **Estabilidad:** Convergencia hacia políticas consistentes y robustas.

4. **Eficiencia Computacional:** El entrenamiento paralelo demuestra ser efectivo para acelerar el aprendizaje.

### Potencial para Aplicaciones Reales

Los resultados sugieren que SAC es un algoritmo promisorio para:
- Tareas de manipulación robótica complejas
- Sistemas con espacios de acción de alta dimensionalidad
- Aplicaciones que requieren control preciso y coordinado

---

## Trabajo Futuro

### Extensiones Recomendadas

1. **Entrenamiento Extendido:**
   - Incrementar significativamente el número de episodios (10,000+)
   - Explorar el potencial completo de aprendizaje del agente

2. **Técnicas Avanzadas:**
   - Implementar Hindsight Experience Replay (HER) para mejorar eficiencia de muestreo
   - Explorar curriculum learning para progresión gradual de dificultad

3. **Evaluación Robusta:**
   - Pruebas con variaciones en configuraciones de puerta
   - Evaluación de transferencia a tareas similares

4. **Optimizaciones:**
   - Explorar arquitecturas de red más sofisticadas
   - Investigar técnicas de regularización para mejorar generalización

### Impacto Potencial

Este trabajo contribuye al avance del aprendizaje por refuerzo en robótica, específicamente en:
- **Manipulación diestra:** Desarrollo de habilidades motoras finas en robots
- **Automatización doméstica:** Capacidades para tareas cotidianas
- **Robótica de servicio:** Aplicaciones en entornos humanos

La implementación exitosa de SAC en esta tarea compleja demuestra la viabilidad de aplicar deep reinforcement learning a problemas reales de manipulación robótica, sentando las bases para futuros avances en el campo.

# Información Técnica y Recursos

## Resumen de Archivos Generados

Este notebook genera los siguientes archivos durante el entrenamiento y evaluación:

### Modelos Entrenados
- `models/sac_parallel/sac_best.zip` - Mejor modelo basado en evaluaciones
- `models/sac_parallel/sac_final.zip` - Modelo al final del entrenamiento
- `models/sac_parallel/sac_checkpoint_*.zip` - Checkpoints periódicos

### Datos de Entrenamiento
- `logs/sac_parallel/training_log.csv` - Métricas detalladas de entrenamiento
- `logs/sac_parallel/training_history.json` - Historia completa de entrenamiento
- `models/sac_parallel/training_info.json` - Información de configuración

### Logs de TensorBoard
- `runs/sac_parallel/` - Logs para visualización en TensorBoard

## Comandos Útiles

```bash
# Visualizar logs de TensorBoard
tensorboard --logdir=runs/sac_parallel

# Verificar archivos generados
ls -la models/sac_parallel/
ls -la logs/sac_parallel/
```

## Requisitos del Sistema

- **RAM:** Mínimo 8GB, recomendado 16GB+
- **GPU:** Opcional pero recomendada para acelerar entrenamiento
- **Tiempo de entrenamiento:** 8-12 horas para entrenamiento completo
- **Espacio en disco:** ~1GB para modelos y logs

## Enlaces y Referencias

- [Gymnasium Robotics - Adroit Hand Door](https://robotics.farama.org/envs/adroit_hand/adroit_door/)
- [Stable Baselines3 - SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html)
- [Paper Original SAC](https://arxiv.org/abs/1801.01290)
- [Adroit Manipulation Platform](https://arxiv.org/abs/1709.10087)

---

**Fin del Notebook - Adroit Hand Door con SAC** 🎉